In [19]:
import pandas as pd
import numpy as np

In [20]:
file = "AnalyticsQuestionnairePitchData.csv"
df = pd.read_csv(file)

## <b>Quick Data file review</b>

In [21]:
df.head()

,PitchId,GamePk,PitcherHand,PitchCall,PitchType,BatterId,PitcherId,BatterSide,Inning,IsTop,...,TrajectoryPolynomialZ2,TrajectoryX0,TrajectoryX1,TrajectoryX2,TrajectoryV0,TrajectoryV1,TrajectoryV2,TrajectoryA0,TrajectoryA1,TrajectoryA2
0,231,1,L,called_strike,CH,19,11,R,1,1,...,-14.281734,1.629367,50.002632,5.681500,-5.673969,-120.768692,-3.060989,8.499877,21.381056,-28.563467
1,384,1,L,ball,CU,19,11,R,1,1,...,-19.321169,1.694219,50.001293,5.523830,-6.150516,-115.130562,-1.678751,-2.671943,17.934855,-38.642338
2,616,1,L,field_out,SL,19,11,R,1,1,...,-13.228232,1.853305,50.000290,5.531102,-5.501207,-127.250221,-1.467974,0.618823,21.358961,-26.456465
3,503,1,L,ball,SI,24,11,L,1,1,...,-12.885942,1.891533,50.001602,5.475145,-4.826164,-133.365570,-2.412576,14.973106,24.863359,-25.771883
4,322,1,L,swinging_strike,CU,24,11,L,1,1,...,-18.384531,1.784340,50.002003,5.493305,-5.491218,-116.880058,-2.260371,0.099588,18.352825,-36.769062


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 67 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   PitchId                            620 non-null    int64  
 1   GamePk                             620 non-null    int64  
 2   PitcherHand                        620 non-null    object 
 3   PitchCall                          620 non-null    object 
 4   PitchType                          601 non-null    object 
 5   BatterId                           620 non-null    int64  
 6   PitcherId                          620 non-null    int64  
 7   BatterSide                         620 non-null    object 
 8   Inning                             620 non-null    int64  
 9   IsTop                              620 non-null    int64  
 10  Balls                              620 non-null    int64  
 11  Strikes                            620 non-null    int64  

<b>It looks like the home and away teams in game 2 were given ID = 1. Determining  the correct assignment below</b>

In [23]:
#determining if home or away is ID=1 in game 2
teamOneBatterIds = df[(df['GamePk'] == 1) &
                     (df['IsTop'] == 1)]['BatterId'].unique()


gameTwoAwayBatters = df[(df['GamePk'] == 2) &
                         (df['IsTop'] == 1)]['BatterId'].unique()

print("overlapping away batters: " , list(set(teamOneBatterIds) & set(gameTwoAwayBatters)))

# checking if both home teams match using same methods
team2BatterIds = df[(df['GamePk'] == 1) &
                     (df['IsTop'] == 0)]['BatterId'].unique()


gameTwoHomeBatters = df[(df['GamePk'] == 2) &
                         (df['IsTop'] == 0)]['BatterId'].unique()

print("overlapping home batters: " , list(set(team2BatterIds) & set(gameTwoHomeBatters)))


overlapping away batters:  [1, 34, 2, 4, 13, 18, 19, 21, 24, 26]
overlapping home batters:  []


<b>Since the away team in both games has overlapping batterIds the away team in each game is ID=1. And there are no overlapping batters for the home teams in both games.<b>

<uL>
<li>game one and two; away team ID = 1</li>
<li>game one home ID = 2</li>
<li>game two home ID changed to = 3</li>
</ul>

In [24]:
# changing game2 home team ID = 3
df.loc[(df['GamePk'] == 2), 'HomeTeamID'] = 3

## <b> Preparing the data</b>

In [25]:
# func to label hits - it will be used for pitching and hitting
def hits(pitchCall, total_bags=False, pos="p"):
    ''' 
        total_bags false returns if hit or not
        total_bags true returns total bases
        
        pos is either p or b. Pitcher or Batter - field errors are not included in pitcher hit total
    '''
    if total_bags:
        if pitchCall == 'single':
            return 1
        elif pitchCall == 'double':
            return 2
        elif  pitchCall == 'triple':
            return 3
        elif pitchCall == 'home_run':
            return 4
        else:
            return 0

    else:
        if pos == 'p':
            if (pitchCall == 'single') | (pitchCall == 'double') |  (pitchCall== 'triple') | (pitchCall == 'home_run'):
                return 1
            else:
                return 0
        else:
            if (pitchCall == 'single') | (pitchCall == 'double') |  (pitchCall== 'triple') | (pitchCall == 'home_run') | (pitchCall== 'field_error'):
                return 1
            else:
                return 0
    
# func to label outs for pitchers
def outs(pitchCall):
    if (pitchCall == 'field_out') | (pitchCall == 'force_out') | (pitchCall == 'strikeout'): 
        return 1
    elif pitchCall == 'grounded_into_double_play':
        return 2
    else:
        return 0
    
# func to boolean for steals
def steals(pitchCall):
    if (pitchCall == 'stolen_base_3b') | (pitchCall == 'stolen_base_2b') | (pitchCall == 'stolen_base_1b'):
        return 1
    else:
        return 0
    
# func to return boolean for type of hit
def extraBases(pitchCall, hit_type):
    '''
        hit type can be 's', 'd', 't', 'hr' to return if the hit was of the specific type
    '''
    if hit_type == 's' and pitchCall == 'single':
        return 1
    elif hit_type == 'd' and pitchCall == 'double':
        return 1
    elif hit_type == 't' and pitchCall == 'triple':
        return 1
    elif hit_type == 'hr' and pitchCall == 'home_run':
        return 1
    else:
        return 0


# creating the  values that will be used to calc. stats
df['p_hits'] = df['PitchCall'].apply(lambda x: hits(x))
df['b_hits'] = df['PitchCall'].apply(lambda x: hits(x, pos='b'))
df['outs'] = df['PitchCall'].apply(outs)
df['strike'] = np.where(df['PitchCall'].str.contains("strike",case=False ), 1, 0)
df['ball'] = np.where(df['PitchCall'].str.contains("ball",case=False ), 1, 0)
df['swstr'] = np.where(df['PitchCall'].str.contains("swinging_strike",case=False ), 1, 0)
df['bb'] = np.where(df['PitchCall'].str.contains("walk",case=False ), 1, 0)
df['k'] = np.where(df['PitchCall'].str.contains("strikeout",case=False ), 1, 0)
df['bases'] = df['PitchCall'].apply(lambda x: hits(x, total_bags=True))
df['stealSurrendered'] = df['PitchCall'].apply(lambda x: steals(x))
df['singles'] = df['PitchCall'].apply(lambda x: extraBases(x, 's'))
df['doubles'] = df['PitchCall'].apply(lambda x: extraBases(x, 'd'))
df['triples'] = df['PitchCall'].apply(lambda x: extraBases(x, 't'))
df['hr'] = df['PitchCall'].apply(lambda x: extraBases(x, 'hr'))
df['sac'] = np.where(df['PitchCall'].str.contains('sac_', case=False), 1, 0)
df['hbp'] = 0
df['batterTeamId'] = np.where(df['IsTop'] == 1, df['AwayTeamID'], df['HomeTeamID'])
df['pitcherTeamId'] = np.where(df['IsTop'] == 1, df['HomeTeamID'], df['AwayTeamID'])

## <b> Analysis</b>

### Batters

In [26]:
df_bats = df.groupby(['GamePk','batterTeamId', 'BatterId']).agg({
                                                  'b_hits':'sum',
                                                  'AtBatNumber':'nunique',
                                                  'bb':'sum',
                                                  'k':'sum',
                                                  'bases':'sum',
                                                  'sac':'sum',
                                                  'singles':'sum',
                                                  'doubles':'sum',
                                                  'triples':'sum',
                                                  'hr':'sum',
                                                  'hbp':'sum'
                                                })

df_bats = df_bats.rename(columns={'b_hits':'hits',
                                  'AtBatNumber':'ab'
                                 })

df_bats['avg'] = df_bats['hits'] / (df_bats['ab'] - df_bats['bb'])
df_bats['obp'] = (df_bats['hits'] + df_bats['bb'] + df_bats['hbp']) / (df_bats['ab'] + df_bats['bb'] + df_bats['hbp'] + df_bats['sac'])
df_bats['slg'] = (df_bats['singles'] + 
                  (df_bats['doubles'] * 2) + 
                  (df_bats['triples'] * 3) + 
                  (df_bats['hr'] * 4)
                ) / df_bats['ab']
df_bats['ops'] = (
                  (df_bats['ab'] * (df_bats['hits'] + df_bats['bb'] + df_bats['hbp'])) +
                  (df_bats['bases'] * (df_bats['ab'] + df_bats['bb'] + df_bats['sac'] + df_bats['hbp']))
                 ) / (df_bats['ab'] * (df_bats['ab'] + df_bats['bb'] + df_bats['sac'] + df_bats['hbp']))

df_bats['contactPct'] = (df_bats['ab'] - df_bats['k']) / df_bats['ab']

df_bats = df_bats.reset_index()

### Pitchers

In [27]:
### pitch count
df_pitches = df.groupby(['GamePk', 'pitcherTeamId', 'PitcherId']).agg({'PitchId':'count',
                                                     'ReleaseSpeed':'mean',
                                                     'ReleaseSpinRate':'mean',
                                                     'strike':'sum',
                                                     'ball':'sum',
                                                     'swstr':'sum',
                                                     'bb':'sum',
                                                      'k':'sum',
                                                      'outs':'sum',
                                                      'p_hits':'sum',
                                                     'BatterId':'nunique',
                                                     'stealSurrendered':'sum'
                                                    })
df_pitches = df_pitches.rename(columns={'PitchId':'totalPitches', 
                                        'BatterId':'battersFaced', 
                                        'bb':'walks',
                                        'p_hits':'hits'
                                       })

# calculating strike %
df_pitches['strikePct'] = df_pitches['strike'] / df_pitches['totalPitches']
df_pitches['swstrPct'] = df_pitches['swstr'] / df_pitches['totalPitches']
df_pitches['ballPct'] = df_pitches['ball'] / df_pitches['totalPitches']
df_pitches['ip'] = df_pitches['outs'].apply(lambda x: x / 3)
df_pitches['k9'] = (9 *  df_pitches['k']) /  df_pitches['ip']



#flattening the grouped data frame for 
df_pitches = df_pitches.reset_index()

## <b>Player Performance</b>

In [28]:
# setting dataframes
g1_pitch = df_pitches[df_pitches['GamePk'] == 1]
g2_pitch = df_pitches[df_pitches['GamePk'] == 2]
g1_bat = df_bats[df_bats['GamePk'] == 1]
g2_bat = df_bats[df_bats['GamePk'] == 2]

#### Pitching stats

In [29]:
# game 1, team 1 - Pitching 
g1_pitch[g1_pitch['pitcherTeamId'] == 1][['totalPitches','PitcherId','strikePct', 'k', 'ip', 'k9']].sort_values('strikePct', ascending= False)

,totalPitches,PitcherId,strikePct,k,ip,k9
3,14,6,0.428571,2,1.000000,18.0
6,11,16,0.272727,1,0.333333,27.0
0,105,1,0.257143,6,5.000000,10.8
5,15,13,0.200000,0,0.666667,0.0
1,18,3,0.166667,2,1.000000,18.0
2,14,4,0.142857,0,1.000000,0.0
4,10,8,0.000000,0,1.000000,0.0


In [30]:
# game 1, team 2 - Pitching 
g1_pitch[g1_pitch['pitcherTeamId'] == 2][['totalPitches','PitcherId','strikePct', 'k', 'ip', 'k9']].sort_values('strikePct', ascending= False)

,totalPitches,PitcherId,strikePct,k,ip,k9
7,10,2,0.400000,2,1.0,18.0
10,20,12,0.350000,1,1.0,9.0
9,99,11,0.232323,5,6.0,7.5
8,29,5,0.206897,0,1.0,0.0
11,16,14,0.062500,0,1.0,0.0


In [31]:
# game 2, team 1 - Pitching 
g2_pitch[g2_pitch['pitcherTeamId'] == 1][['totalPitches','PitcherId','strikePct', 'k', 'ip', 'k9']].sort_values('strikePct', ascending= False)

,totalPitches,PitcherId,strikePct,k,ip,k9
13,6,3,0.500000,1,0.333333,27.0
14,9,6,0.444444,0,1.000000,0.0
12,96,1,0.281250,7,7.000000,9.0
15,15,17,0.200000,0,0.666667,0.0


In [32]:
# game 2, team 3 - Pitching 
g2_pitch[g2_pitch['pitcherTeamId'] == 3][['totalPitches','PitcherId','strikePct', 'k', 'ip', 'k9']].sort_values('strikePct', ascending= False)

,totalPitches,PitcherId,strikePct,k,ip,k9
17,22,9,0.363636,3,1.000000,27.000000
19,17,15,0.294118,2,1.000000,18.000000
16,77,7,0.246753,5,5.666667,7.941176
18,17,10,0.235294,0,1.000000,0.000000


#### Batting Stats

In [33]:
# game 1, team 1 - batting
g1_bat[g1_bat['batterTeamId'] == 1][['hits', 'ab', 'bb', 'avg', 'ops', 'contactPct']].sort_values('ops', ascending=False)

,hits,ab,bb,avg,ops,contactPct
9,4,5,0,0.800000,1.600000,1.000000
7,3,5,1,0.750000,1.266667,1.000000
4,2,5,0,0.400000,1.200000,0.800000
3,1,5,0,0.200000,1.000000,1.000000
11,2,5,1,0.500000,0.900000,0.800000
5,1,3,0,0.333333,0.666667,0.666667
6,1,5,0,0.200000,0.400000,0.600000
0,0,1,0,0.000000,0.000000,1.000000
1,0,2,0,0.000000,0.000000,0.000000
2,0,5,0,0.000000,0.000000,1.000000


In [34]:
# game 1, team 2 - batting
g1_bat[g1_bat['batterTeamId'] == 2][['hits', 'ab', 'bb', 'avg', 'ops', 'contactPct']].sort_values('ops', ascending=False)

,hits,ab,bb,avg,ops,contactPct
13,1,1,0,1.00,2.000000,1.00
22,3,4,0,0.75,1.750000,0.75
18,1,2,1,1.00,1.166667,1.00
17,1,5,0,0.20,1.000000,1.00
14,2,5,0,0.40,0.800000,0.80
16,1,5,1,0.25,0.533333,0.80
24,0,1,1,NaN,0.500000,1.00
15,0,1,0,0.00,0.000000,0.00
19,0,3,0,0.00,0.000000,1.00
20,0,4,0,0.00,0.000000,0.25


In [35]:
# game 2, team 1 - batting
g2_bat[g2_bat['batterTeamId'] == 1][['hits', 'ab', 'bb', 'avg', 'ops', 'contactPct']].sort_values('ops', ascending=False)

,hits,ab,bb,avg,ops,contactPct
25,1,1,0,1.000000,2.00,1.000000
27,2,4,0,0.500000,1.25,1.000000
34,1,4,1,0.333333,1.15,1.000000
31,2,5,0,0.400000,0.80,0.800000
30,2,5,0,0.400000,0.60,0.600000
28,0,4,1,0.000000,0.20,0.250000
29,0,4,1,0.000000,0.20,0.750000
33,1,4,0,0.250000,0.20,0.750000
26,0,3,0,0.000000,0.00,0.333333
32,0,5,0,0.000000,0.00,1.000000


In [36]:
# game 2, team 3 - batting
g2_bat[g2_bat['batterTeamId'] == 3][['hits', 'ab', 'bb', 'avg', 'ops', 'contactPct']].sort_values('ops', ascending=False)

,hits,ab,bb,avg,ops,contactPct
40,2,4,0,0.50,0.750000,0.750000
44,1,4,0,0.25,0.750000,0.750000
36,1,4,0,0.25,0.500000,0.750000
35,0,4,3,0.00,0.428571,1.000000
37,0,2,0,0.00,0.000000,0.500000
38,0,1,0,0.00,0.000000,0.000000
39,0,3,0,0.00,0.000000,0.333333
41,0,4,0,0.00,0.000000,0.750000
42,0,3,0,0.00,0.000000,1.000000
43,0,3,0,0.00,0.000000,1.000000
